# Simulation of a crack growth in a thick square plate based on Peridynamic Theory

Silling, S. A., & Askari, E. (2005). *A meshfree method based on the peridynamic model of solid mechanics*. *Computers & Structures, 83*(17-18), 1526-1535. https://doi.org/10.1016/j.compstruc.2004.11.026

## Setup

### Setup Python Modules

In this section, we will import the python libraries:

- **NumPy**: This library is fundamental for numerical computations in Python. It provides support for arrays, matrices, and many mathematical functions that operate on these data structures.
- **Matplotlib**: A plotting library for creating static, animated, and interactive visualizations in Python. It's particularly useful for generating plots, histograms, and other types of charts.

In [56]:
import numpy as np  # Importing the NumPy library for numerical operations and handling arrays
import matplotlib.pyplot as plt  # Importing the Pyplot module from Matplotlib for plotting and visualizations
import pandas as pd
import seaborn as sns

## Parameters

### Consistency of Units in Calculations

To avoid errors in our calculations, we will adhere to a consistent set of units for the following physical quanities:

- Length/Distance: millimeters (mm)
- Mass: kilograms (kg)
- Time: microseconds (μs)

In [2]:
rho = 8e-6 #material density
Young_modulus = 19578.55  # Everything in kg and mm
nu = 0.3 #Poisson's Ratio
K = Young_modulus/(3.0*(1-2.0*nu))
horizon = 1.6
s0 = 0.02
total_time = 15
del_T = 0.5
time_steps = int(total_time/del_T)

<img src="fig5.png" alt="Velocity applied to plate horizontal edges" width="600">

In [3]:
u_applied_05 = np.linspace(0.0,2e4,time_steps//3)
u_applied_510 = np.linspace(2e4,0,time_steps//3)
u_applied_1015 = np.linspace(0,0,time_steps//3)
u_applied = np.concatenate((u_applied_05, u_applied_510, u_applied_1015))

## Generating Meshfree Geometry

In [4]:
def Create_geometry():
    # Define the number of divisions along the x and y axes
    nx = 50 # Number of divisions in the x-direction
    ny = 50 # Number of divisions in the y-direction

     # Define the minimum and maximum coordinates for x and y axes
    x_min = 2.0     # Minimum x-coordinate (in mm)
    x_max = 48.0    # Maximum x-coordinate (in mm)
    y_min = 2.0     # Minimum y-coordinate (in mm)
    y_max = 48.0    # Maximum y-coordinate (in mm)

    # Generate linearly spaced coordinates for x and y axes
    x_coord = np.linspace(x_min,x_max, nx)  # x-coordinates
    y_coord = np.linspace(y_min, y_max, ny) # y-coordinates

    # Calculate the total number of nodes
    nnodes = nx*ny # Total number of nodes in the grid

    # Initialize an array to hold the coordinates of all nodes
    coord_array = np.zeros((nnodes,2)) # Array to store x and y coordinates of each node

    # Calculate the volume of the node, assuming a thickness of 1 mm
    vol = (x_coord[1] - x_coord[0]) * (y_coord[1] - y_coord[0]) * 1.0 

     # Initialize lists to store the indices of top and bottom nodes
    top_nodes = []
    bottom_nodes = []
    count = 0

    # Loop through all the nodes and identify boundary nodes
    for i in range(nx):
        for k in range(ny):
            coord_array[count,0] = x_coord[k] # Fetch x-coordinate of current node
            coord_array[count, 1] = y_coord[i] # Fetch x-coordinate of current node
            
            # Identify and store nodes on the top boundary (y = y_max)
            if y_coord[i] == y_max:
                top_nodes.append(count)

            # Identify and store nodes on the bottom boundary (y = y_min)
            if y_coord[i] == y_min:
                bottom_nodes.append(count)
            count = count + 1

    # Return the coordinate array, top and bottom node lists, total number of nodes, and node volume
    return coord_array, top_nodes, bottom_nodes, nnodes, vol

In [5]:
coord_array, top_nodes, bottom_nodes, nnodes, vol = Create_geometry()

## Neighborsearch

We need to find for each node $x_p$ the neighborhood $H_\delta(x_p)$ such that $H_\delta(x_p)=\lbrace i | \Vert x_i - x_p \Vert \leq \delta \rbrace$.

In [6]:
def Get_nodes_in_horizon(coord_array, nnodes, horizon):

    # Initialize an array to store the indices of nodes within the horizon for each node
    nodal_array = np.zeros((nnodes, 50)).astype(int)

    # Loop over all nodes to find neighbors within the horizon
    for i in range(nnodes):
        count = 0

        # Loop over all nodes to check their distance from the current node i
        for k in range(nnodes):

            if k!=i: # Skip the comparison with the same node
                x_0 = coord_array[i,0]    # fetch x-coordinate of node i
                y_0 = coord_array[i,1]    # fetch y-coordinate of node i
                x_1 = coord_array[k,0]    # fetch x-coordinate of node k
                y_1 = coord_array[k,1]    # fetch y-coordinate of node k
                R = horizon

                # Calculate the distance between nodes i and k
                dist = (x_1 - x_0)**2.0 + (y_1 - y_0)**2.0

                # Check if node k is within the horizon of node i
                if dist <= R**2.0:
                    nodal_array[i,count] = k  # Store the index of node k in the nodal array for node i
                    count = count + 1 

    return nodal_array

In [7]:
nodal_connectivity = Get_nodes_in_horizon(coord_array, nnodes, horizon)

In [14]:
# Initialize an array to store displacement vectors for each node at every time step
u_array_store = np.zeros((time_steps,nnodes,2))

# Initialize an array to store the current displacement vector for each node
u_array_current = np.zeros_like(coord_array)

current_time = 0


for n in range(time_steps):
    print('Time step:     ', n)
    
    # Increment the current time by the time step size
    current_time = current_time + del_T

    # Apply displacement to top and bottom nodes according to the applied displacement at this time step
    u_array_current[top_nodes,1] = u_applied[n]
    u_array_current[bottom_nodes,1] = -u_applied[n]

    # Loop over each node to calculate the new displacement based on nodal connectivity
    for i in range(nnodes):

        # Identify connected nodes for the current node i
        connected_nodes = np.nonzero(nodal_connectivity[i, :])[0]

        # Ensure that the connectivity includes node i itself if it has no connections
        if nodal_connectivity[i,0]==0:
            np.concatenate((np.zeros((1,)), connected_nodes))

        # Initialize sums for calculating acceleration components
        sum_term_x = 0.0
        sum_term_y = 0.0

        # Loop over all connected nodes to
        for p in range(len(connected_nodes)):

            # Calculate relative displacement (eta) between node i and connected node p
            eta_x = u_array_current[nodal_connectivity[i,connected_nodes[p]],0] - u_array_current[i,0]
            eta_y = u_array_current[nodal_connectivity[i,connected_nodes[p]],1] - u_array_current[i,1]

            # Calculate relative position vector (ksi) between node i and connected node p
            ksi_x = coord_array[nodal_connectivity[i,connected_nodes[p]],0] - coord_array[i,0]
            ksi_y = coord_array[nodal_connectivity[i,connected_nodes[p]],1] - coord_array[i,1]

            # Calculate norms (magnitudes) of eta and ksi
            eta_norm = np.sqrt(eta_x**2.0 + eta_y**2.0)
            ksi_norm = np.sqrt(ksi_x ** 2.0 + ksi_y ** 2.0)

            # Calculate eta + ksi vector and its norm
            eta_x_plus_ksi_x = eta_x + ksi_x
            eta_y_plus_ksi_y = eta_y + ksi_y
            eta_plus_ksi_norm = np.sqrt(eta_x_plus_ksi_x**2.0 + eta_y_plus_ksi_y**2.0)
            
            # Calculate the material constant c based on bulk modulus K and horizon distance    
            c = (18.0 * K) / (np.pi * horizon ** 4.0)

            # Calculate the stretch s
            s = ( eta_plus_ksi_norm - ksi_norm ) / ksi_norm

            # Determine the damage factor mu based on a threshold s0
            if s>s0:
                mu = 0.0 # Set mu to 0 if stretch exceeds threshold, representing damage
            else:
                mu = 1.0 # Set mu to 1 if no damage occurs

            # Calculate the contribution to acceleration in x and y directions
            u_double_dot_x_temp = (eta_x_plus_ksi_x/eta_plus_ksi_norm) * c * s * mu
            u_double_dot_y_temp = (eta_y_plus_ksi_y/eta_plus_ksi_norm) * c * s * mu

            # Sum up the contributions from all connected nodes
            sum_term_x = sum_term_x + u_double_dot_x_temp*vol
            sum_term_y = sum_term_y + u_double_dot_y_temp*vol

        # Calculate the acceleration components for node i
        u_double_dot_x = sum_term_x/rho
        u_double_dot_y = sum_term_y/rho